In [20]:
from rdflib import Graph
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import random
import numpy as np

In [23]:
g = Graph()
#g.parse("sto/sto-enriched-relatedTo.nt", format="nt")
g.parse('sto/transitive_closure/transitive.nt', format="nt")
qres = g.query(
    """PREFIX owl: <http://www.w3.org/2002/07/owl#>
       PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
       PREFIX sto: <https://w3id.org/i40/sto#>
    
    select distinct ?s where {
            ?s ?p ?o .
    }""")
        
entities = pd.DataFrame(columns=['s'])
obj = []
for row in qres:
    obj.append(str("%s" %row))
entities.s=obj

In [24]:
entities.shape

(249, 1)

# Check number of independent cluster

In [25]:
cosine_matrix = pd.DataFrame(index=entities.s.tolist(), columns=entities.s.tolist())
cosine_matrix = cosine_matrix.fillna(0)

g = Graph()
#g.parse("sto/sto-enriched-relatedTo.nt", format="nt")
g.parse('sto/transitive_closure/transitive.nt', format="nt")

prop = 'relatedTo'
list_pd = pd.DataFrame(columns=['s', 'p', 'o'])
for i in range(entities.shape[0]):
    standard = entities.s[i]
    qres = g.query(
        """PREFIX owl: <http://www.w3.org/2002/07/owl#>
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            PREFIX sto: <https://w3id.org/i40/sto#>
            select distinct ?o where {
                <""" +standard+ """> sto:""" +prop+ """ ?o .
            }""")
    lst = pd.DataFrame(columns=['s', 'p', 'o'])
    obj = []
    for row in qres:
        obj.append(str("%s" %row))
    
    if len(obj)>0:
        for col in obj:
            cosine_matrix.at[standard, col] = 1

In [27]:
cosine_matrix.head(15)

,https://w3id.org/i40/sto#ISO_29100,https://w3id.org/i40/sto#ISO_27019,https://w3id.org/i40/sto#IEC_61987_X,https://w3id.org/i40/sto#NIST_800-146,https://w3id.org/i40/sto#DIN_EN_50159,https://w3id.org/i40/sto#IEC_61850_P7_S2_E1,https://w3id.org/i40/sto#SCOR,https://w3id.org/i40/sto#IEC_62061,https://w3id.org/i40/sto#IEC_61850_P7_S2_E2,https://w3id.org/i40/sto#IEC_61968_P3_E1,...,https://w3id.org/i40/sto#IEC_61691_P6_E1,https://w3id.org/i40/sto#IEC_60286_P5_E3,https://w3id.org/i40/sto#ETSI_ULE,https://w3id.org/i40/sto#IEC_61010_P2_S202_E1,https://w3id.org/i40/sto#IEC_61310_P1_E2,https://w3id.org/i40/sto#ISO_20140_5,https://w3id.org/i40/sto#IEC_60864_P1_E1,https://w3id.org/i40/sto#IEC_61010_P2_S201_E2,https://w3id.org/i40/sto#W3C_SPARQL,https://w3id.org/i40/sto#MTConnect
https://w3id.org/i40/sto#ISO_29100,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
https://w3id.org/i40/sto#ISO_27019,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
https://w3id.org/i40/sto#IEC_61987_X,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
https://w3id.org/i40/sto#NIST_800-146,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
https://w3id.org/i40/sto#DIN_EN_50159,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
https://w3id.org/i40/sto#IEC_61850_P7_S2_E1,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
https://w3id.org/i40/sto#SCOR,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
https://w3id.org/i40/sto#IEC_62061,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
https://w3id.org/i40/sto#IEC_61850_P7_S2_E2,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
https://w3id.org/i40/sto#IEC_61968_P3_E1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [28]:
# connected components in an undirected graph 
class Graph: 
      
    # init function to declare class variables 
    def __init__(self,V): 
        self.V = V 
        self.adj = [[] for i in range(V)] 
  
    def DFSUtil(self, temp, v, visited): 
  
        # Mark the current vertex as visited 
        visited[v] = True
  
        # Store the vertex to list 
        temp.append(v) 
  
        # Repeat for all vertices adjacent 
        # to this vertex v 
        for i in self.adj[v]: 
            if visited[i] == False: 
                  
                # Update the list 
                temp = self.DFSUtil(temp, i, visited) 
        return temp 
  
    # method to add an undirected edge 
    def addEdge(self, v, w): 
        self.adj[v].append(w) 
        self.adj[w].append(v) 
  
    # Method to retrieve connected components
    # in an undirected graph
    def connectedComponents(self):
        visited = []
        cc = []
        for i in range(self.V):
            visited.append(False)
        for v in range(self.V):
            if visited[v] == False:
                temp = []
                cc.append(self.DFSUtil(temp, v, visited))
        return cc

In [29]:
array = np.arange(0, 249).tolist()
cosine_matrix.index =array
cosine_matrix.head(10)

,https://w3id.org/i40/sto#ISO_29100,https://w3id.org/i40/sto#ISO_27019,https://w3id.org/i40/sto#IEC_61987_X,https://w3id.org/i40/sto#NIST_800-146,https://w3id.org/i40/sto#DIN_EN_50159,https://w3id.org/i40/sto#IEC_61850_P7_S2_E1,https://w3id.org/i40/sto#SCOR,https://w3id.org/i40/sto#IEC_62061,https://w3id.org/i40/sto#IEC_61850_P7_S2_E2,https://w3id.org/i40/sto#IEC_61968_P3_E1,...,https://w3id.org/i40/sto#IEC_61691_P6_E1,https://w3id.org/i40/sto#IEC_60286_P5_E3,https://w3id.org/i40/sto#ETSI_ULE,https://w3id.org/i40/sto#IEC_61010_P2_S202_E1,https://w3id.org/i40/sto#IEC_61310_P1_E2,https://w3id.org/i40/sto#ISO_20140_5,https://w3id.org/i40/sto#IEC_60864_P1_E1,https://w3id.org/i40/sto#IEC_61010_P2_S201_E2,https://w3id.org/i40/sto#W3C_SPARQL,https://w3id.org/i40/sto#MTConnect
0,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
5,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
6,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
7,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
8,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [30]:
graph = Graph(249);
column = cosine_matrix.columns
for i in range(len(column)):
    index = cosine_matrix.loc[cosine_matrix[column[i]] == 1].index.tolist()
    if len(index)>0:
        for j in index:
            graph.addEdge(j, i)

In [33]:
cc = graph.connectedComponents()
print("Following are connected components")
print((cc))
count=0
for k in cc:
    if len(k)>1:
        print(len(k))
        count+=len(k)
    else:
        print('desconected: '+ str(k))

print(count)

Following are connected components
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 89, 90, 91, 92, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 118, 119, 120, 121, 123, 124, 125, 126, 127, 128, 129, 130, 132, 133, 135, 137, 138, 140, 141, 142, 144, 145, 148, 149, 150, 151, 154, 155, 156, 157, 158, 163, 166, 168, 169, 172, 184], [9, 194, 197, 205, 213, 216, 223, 234], [41, 147, 190, 212, 214, 227], [49, 215, 217, 228], [64, 88, 165, 189, 198, 226, 232], [93, 117, 122, 136, 146, 152, 153, 160, 161, 162, 164, 167, 170, 173, 174, 175, 176, 178, 179, 180, 191, 192, 196, 199], [131, 134, 139, 143, 159, 171, 177, 182, 183, 186, 188, 193, 201, 209], [181,

In [9]:
#standard = "https://w3id.org/i40/sto#RFC_1122"
standard = "?s"
prop = 'relatedTo'
qres = g.query(
    """PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX sto: <https://w3id.org/i40/sto#>
        select distinct ?s where {
            """ +standard+ """ sto:""" +prop+ """ ?o .
        }""")
sto = []
for row in qres:
    sto.append(str("%s" %row))
len(sto)

232

In [12]:
qres = g.query(
    """PREFIX sto: <https://w3id.org/i40/sto#>
        select ?sto1 where {
            ?sto1 sto:relatedTo ?sto2 .
            ?sto2 sto:relatedTo ?sto3 .
            ?sto1 sto:relatedTo ?sto3 .
        }""")

sto = []
for row in qres:
    sto.append(str("%s" %row))
print('firtst query: ', len(sto))

qres = g.query(
    """PREFIX sto: <https://w3id.org/i40/sto#>
        select ?sto1 where {
            ?sto1 sto:relatedTo ?sto2 .
            ?sto2 sto:relatedTo ?sto3 .
        }""")
sto = []
for row in qres:
    sto.append(str("%s" %row))
print('second query: ', len(sto))

firtst query:  61517
second query:  84775
